We compare minimum average particle distance acorss trajecotry, last-step average particle distance across trajecotries, minimum average semantic distance across trajecotries, last-step average particle distance across trajectories, max_IoU agains the goal, automatically detected SR. We fix the trajecotry horizon as 20 regardless if it reaches success or not. Because, we also want to test the policies capability not to mess the success states. The evalaution arena should not have readjust pick assistance.

TODO:
1. Regenerate the oracle folding with fixed 20 horizon.
2. Generate the huma policy
3. in hydra train and eval, seperate the train trajectory setup from the eval trajecotry setup, because some algo needs the reajust-pick assistance.
    * use ray-capsulated arena for pyflex.

In [1]:
import os
import pandas as pd
import numpy as np
import ast


# -------------------------------
# HELPER: parse list-like strings
# -------------------------------

def parse_list(s):
    """Convert string like '[0.2, 0.1, 0.05]' into list of floats."""
    if isinstance(s, list):
        return s
    return ast.literal_eval(s)


# -------------------------------
# METRIC EXTRACTION LOGIC

def aggregate_performance(csv_path):
    df = pd.read_csv(csv_path)

    # Convert list-like string columns to actual lists
    df["particle_dist"] = df["evaluation/mean_particle_distance"].apply(parse_list)
    df["semantic_dist"] = df["evaluation/semantic_keypoint_distance"].apply(parse_list)
    df["iou"] = df["evaluation/max_IoU"].apply(parse_list)
    df["iou_flat"] = df["evaluation/max_IoU_to_flattened"].apply(parse_list)
    df["nc"] = df["evaluation/normalised_coverage"].apply(parse_list)
    df["success_list"] = df["evaluation/success"].apply(parse_list)

    # ---------------------------
    # Extract per-row statistics
    # ---------------------------

    min_particle_distance = [np.min(x) for x in df["particle_dist"]]
    min_semantic_distance = [np.min(x) for x in df["semantic_dist"]]

    max_max_iou = [np.max(x) for x in df["iou"]]
    max_iou_flat = [np.max(x) for x in df["iou_flat"]]
    max_nc = [np.max(x) for x in df["nc"]]

    last_particle_distance = [x[-1] for x in df["particle_dist"]]
    last_semantic_distance = [x[-1] for x in df["semantic_dist"]]
    last_max_iou = [x[-1] for x in df["iou"]]

    # ---------------------------
    # Success + steps-to-finish
    # ---------------------------

    first_success_flags = []
    final_success_flags = []
    steps_to_first_success = []

    for success_list in df["success_list"]:
        if any(success_list):
            first_success_flags.append(1)
            steps_to_first_success.append(success_list.index(True))
        else:
            first_success_flags.append(0)
            steps_to_first_success.append(len(success_list) - 1)
        if success_list[-1]:
            final_success_flags.append(1)

    # ---------------------------
    # Aggregated metrics (means)
    # ---------------------------

    min_particle_mean = np.mean(min_particle_distance)
    last_particle_mean = np.mean(last_particle_distance)

    min_semantic_mean = np.mean(min_semantic_distance)
    last_semantic_mean = np.mean(last_semantic_distance)

    max_iou_mean = np.mean(max_max_iou)
    max_iou_flat_mean = np.mean(max_iou_flat)
    last_iou_mean = np.mean(last_max_iou)
    max_nc_mean = np.mean(max_nc)

    first_success_rate = np.mean(first_success_flags)
    final_success_rate = np.mean(final_success_flags)

    # ---------------------------
    # Standard deviations
    # ---------------------------

    min_particle_std = np.std(min_particle_distance)
    last_particle_std = np.std(last_particle_distance)

    min_semantic_std = np.std(min_semantic_distance)
    last_semantic_std = np.std(last_semantic_distance)

    max_iou_std = np.std(max_max_iou)
    last_iou_std = np.std(last_max_iou)

    max_nc_std = np.std(max_nc)

    # ---------------------------
    # Episode counts
    # ---------------------------

    num_episodes = len(df)
    first_success_episodes = int(np.sum(first_success_rate))
    final_success_episodes = int(np.sum(final_success_rate))
    steps_to_finish_mean = np.mean(steps_to_first_success)
    steps_to_finish_std = np.std(steps_to_first_success)

    return {
        # Means
        "min_particle_distance_mean": min_particle_mean,
        "last_particle_distance_mean": last_particle_mean,
        "min_semantic_distance_mean": min_semantic_mean,
        "last_semantic_distance_mean": last_semantic_mean,
        "max_IoU_mean": max_iou_mean,
        "max_IoU_to_flattened_mean": max_iou_flat_mean,
        "max_NC_mean": max_nc_mean, 
        "last_IoU_mean": last_iou_mean,
        "steps_to_first_success_mean": steps_to_finish_mean,

        # Standard deviations
        "min_particle_distance_std": min_particle_std,
        "last_particle_distance_std": last_particle_std,
        "min_semantic_distance_std": min_semantic_std,
        "last_semantic_distance_std": last_semantic_std,
        "max_IoU_std": max_iou_std,
        "last_IoU_std": last_iou_std,
        "steps_to_first_success_std": steps_to_finish_std,
        "max_NC_std": max_nc_std, 

        # Others
        "first_success_rate": first_success_rate,
        "final_success_rate": final_success_rate,
        "num_episodes": num_episodes,
        "first_success_episodes": first_success_episodes,
        "final_success_episodes": final_success_episodes
    }

In [2]:
# We evaluate the ability to flattening, ablilty to folding and ability to recover from failed folding states.
# first success in a trajectory --> ability to folding
# final step success in a trajectory --> ability to recognise success and recover from failures.

data_dir = "/mnt/ssd/garment_folding_data"
experiments = [
    {
        "name": "human_multi_primitive_multi_longsleeve_folding_from_crumpled",
        "label": "Human"
    }
]

rows = []

for exp in experiments:
    csv_path = os.path.join(
        data_dir,
        exp["name"],
        "eval_checkpoint_-1",
        "performance.csv"
    )
    print('csv', csv_path)
    if not os.path.exists(csv_path):
        print(f"WARNING: Missing file: {csv_path}")
        continue

    metrics = aggregate_performance(csv_path)
    metrics["experiment"] = exp["label"]
    rows.append(metrics)

results_df = pd.DataFrame(rows)
cols = ["experiment"] + [c for c in results_df.columns if c != "experiment"]
results_df = results_df[cols]
display(results_df)

csv /mnt/ssd/garment_folding_data/human_multi_primitive_multi_longsleeve_folding_from_crumpled/eval_checkpoint_-1/performance.csv


,experiment,min_particle_distance_mean,last_particle_distance_mean,min_semantic_distance_mean,last_semantic_distance_mean,max_IoU_mean,max_IoU_to_flattened_mean,max_NC_mean,last_IoU_mean,steps_to_first_success_mean,...,last_semantic_distance_std,max_IoU_std,last_IoU_std,steps_to_first_success_std,max_NC_std,first_success_rate,final_success_rate,num_episodes,first_success_episodes,final_success_episodes
0,Human,0.063959,0.063959,0.059834,0.059834,0.884821,0.865817,0.951157,0.884821,8.333333,...,0.005295,0.009173,0.009173,0.942809,0.015806,1.0,1.0,3,1,1
